# TODO
* Create a filter for equipments, adding to wo_type and clientid
* Comment and attempt to refactor

# Read data and calculate TTF for each failure mode



In [ ]:
%%html
<script>
code_show=false;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
To hide/show code, click <a href="javascript:code_toggle()">here</a>.

In [ ]:
import sys
sys.path.append(r'..\bin')
import analytics_stat.distributions as di
import analytics_stat.db_mgmt as db
import analytics_stat.extractor as extractor
import pandas as pd
import os
from IPython.display import HTML, Image, display, clear_output
import xlrd
import psycopg2
from sqlalchemy import create_engine, Column, Table, MetaData
from sqlalchemy.sql import text
import numpy as np
from scipy import stats
import re
import time
import ipywidgets as widgets
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def create_selector(wo_table):
    """
    Given a dataframe, returns a display to select from different types of workorders with dropdowns
    """
    wo_type_labels_nano = ['-','PRV','PRD','COR','INS',
                           'CLN','LUB','MET','LAC','COM',
                           'CON','GEN']
    container= list()
    type_selects = list()
    
    l = 0
    for i, label in enumerate(wo_table['wo_type']):
        l += 1
        type_selects.append(widgets.Dropdown(
                         options=wo_type_labels_nano,
                         description=label,
                         #value=wo_table.loc[i,'wo_type_nano'],
                         disabled=False))
        
        if l == len(wo_table):  # number of columns
            container.append(widgets.VBox(type_selects))
            type_selects = list()
            l=0

    return container

class work_orders:
    def __init__(self):
        self._clientid = None
        self._symptom = None
        self._select_wo = None
        self._top_freq_wo = None
        self._ttf = None
        
    def update_clientid(self,args):
        clear_output()
        print('Selected Client:\t{}'.format(args['new']))
    
        if args['new']!='-':
            self._clientid  = reliability_status.loc[reliability_status['name']==select_client.value, 'clientid'].values[0]
            print('Client id:\t\t{}'.format(self._clientid))
        else:
            self._clientid  = None
            print('Client id:\t\t{}'.format('-'))
            
    def update_symptom(self,args):
        clear_output()
        print('Selected Symptom:\t{}'.format(args['new']))
    
        if args['new']!='All':
            self._symptom  = reliability_status.loc[reliability_status['name']==select_client.value, 'clientid'].values[0]
            print('Selected symptom:\t\t{}'.format(self._clientid))
        else:
            self._symptom  = None
            print('Selected symptom:\t\t{}'.format('All'))
            
    def display_widgets(self, widget):
        display(widget)
        
            
    def config_wo(self,args):
        print(self._clientid)
        if self._clientid:
            wo_header = pd.read_sql_query("""SELECT  wo_header_short, wo_type, wo_type_nano 
                                        FROM reliability WHERE clientid = {}; """.format(self._clientid)
                               , engine)
            print(wo_header)
            if wo_header['wo_type'].count()!=0:

                # TODO: Normalize texts to be more efficient in the group by
                self._top_freq_wo = wo_header.groupby(['wo_type','wo_header_short']).size() \
                                .to_frame('count_header').reset_index() \
                                .groupby(['wo_type','count_header'])['wo_header_short'].unique() \
                                .to_frame().reset_index() \
                                .sort_values('count_header', ascending=False) \
                                .drop_duplicates(subset='wo_type')\
                                .sort_values('wo_type')\
                                .reset_index()

                display(self._top_freq_wo.iloc[:, 1:])

                self._select_wo = widgets.HBox(create_selector(self._top_freq_wo),
            #                      layout=widgets.Layout(width='50%', height='10px')
                                 )
                
                self.display_widgets(self._select_wo)
                
            else:
                print('Client has no WO types specified.')


        elif self._clientid is None:
            print('Please, select client.')
            

    def update_wo_call(self, update_wo_types):


        if self._top_freq_wo is not None:
            wo_type_labels = list(self._top_freq_wo['wo_type'].values)
            query_labels = str()
            for i,label in enumerate(wo_type_labels):
                query_labels += "WHEN wo_type IN ('{}') THEN '{}'\n".format(label,self._select_wo.children[0].children[i].value)

            query_update_wo = """UPDATE reliability 
               SET wo_type_nano = 
               CASE 
               {}
               ELSE wo_type_nano 
               END;""".format(query_labels)

    #         print(query_update_wo)

            table_name = 'reliability'

            exists=engine.execute(text("""SELECT * FROM information_schema.tables WHERE table_name = '{}'""".format(table_name)).execution_options(autocommit=True)).fetchall()
            if exists:
                engine.execute(text(query_update_wo).execution_options(autocommit=True))
                
            print('changes were commited.')
        else:
            print('no changes.')
            
    def compute_ttf(self, args):
        if self._clientid:
            if self._symptom:
                raw_data = pd.read_sql_query("""SELECT * FROM reliability WHERE clientid = {} AND symptoms = {}; """.format(self._clientid), engine)
            elif self._symptom is None:
                raw_data = pd.read_sql_query("""SELECT * FROM reliability WHERE clientid = {}; """.format(self._clientid), engine)

            raw_data = raw_data.sort_values(by='wo_type_nano')
            raw_data = raw_data.sort_values(by='wo_start_datetime')
            try:
                raw_data['TTR'] = (raw_data['wo_finish_datetime'] - raw_data['wo_start_datetime'])
                raw_data['TTR (Days)'] = raw_data['TTR'].astype('timedelta64[D]')

                #raw_data['TTF'] = raw_data.loc[(raw_data['Cl.']=='COR1'),'Inic.prog.'].sort_values().diff() - raw_data.loc[(raw_data['Cl.']=='COR1'),'TTR'].values
                raw_data['TTF'] = raw_data.loc[raw_data['wo_type_nano']=='COR','wo_start_datetime'].sort_values().diff() - raw_data.loc[raw_data['wo_type_nano']=='COR','TTR'].values

                raw_data['TTF (Days)'] = raw_data['TTF'].astype('timedelta64[D]')
            except:
                raw_data['TTF'] = raw_data.loc[raw_data['wo_type_nano']=='COR','wo_start_datetime'].sort_values().diff()

                raw_data['TTF (Days)'] = raw_data['TTF'].astype('timedelta64[D]')

            self._ttf = raw_data.loc[raw_data['TTF (Days)']>0,'TTF (Days)'].values
            self._ttf  = self._ttf .ravel().astype(int)
            print(pd.DataFrame(self._ttf).describe().transpose())
            plt.hist(self._ttf)
        
            
    def save_ttf(self, args):
        if self._clientid:
            if len(self._ttf)>=2:  # at lest two points
                if self._symptom:
                    np.savetxt("../data/results/ttf/ttf_symptom{}_client{}.csv".format(self._symptom,self._clientid), self._ttf, delimiter=",")
                elif self._symptom is None:
                    np.savetxt("../data/results/ttf/ttf_symptomAll_client{}.csv".format(self._symptom,self._clientid), self._ttf, delimiter=",")
            else:
                print('Please, compute ttf.')
        
        elif self._clientid is None:
            print('Please, select client.')
            
    def save_json(self, args):
        json_ttf = dict()
        
        if self._clientid:
            if len(self._ttf)>=2:
                if self._symptom:
                    json_ttf['symptom'] = int(self._symptom)
                elif self._symptom is None:
                    json_ttf['symptom'] = None
                
                json_ttf['ttf'] = self._ttf.tolist()
                json_ttf['client_id'] = int(self._clientid)
                json_ttf['client_name'] = select_client.value
                                
                with open("../data/results/ttf/ttf_symptom{}_client{}.json".format(json_ttf['symptom'],
                                                                                   json_ttf['client_id']), 'w') as f:
                    json.dump(json_ttf, f)
                    
                print('JSON saved.')
                
            else:
                print('Please, compute ttf.')
                
        
        elif self._clientid is None:
            print('Please, select client.')    

## Database connection

In [ ]:
engine = db.db_engine()

meta = MetaData()
meta.bind = engine
meta.reflect()

## Status of Reliability table

In [ ]:
# --------------------------#
#     get clients data      #
# ------------------------- #
reliability_status = pd.read_sql_query("""SELECT c.id as clientid, c.name, COALESCE( r.cnt, 0 ) AS n_rows
                                    FROM clients c
                                    LEFT JOIN 
                                    (SELECT clientid, COUNT(*) AS cnt FROM reliability GROUP BY clientid) r
                                    ON r.clientid = c.id
                                    ORDER BY n_rows DESC;"""
                               , engine)
display(reliability_status)

## Select client for analysis

In [ ]:
client_names = ['-'] + list(np.sort(reliability_status.loc[reliability_status['n_rows']>0,'name'].values))

select_client = widgets.Dropdown(
    options=client_names,
    value= '-',
    description='Client:',
    continuous_update=True)


button = widgets.Button(description="Configure WO")
#display(button)

client = work_orders()

button.on_click(client.config_wo)

clients_mgmt = widgets.HBox([select_client])

client_id = select_client.observe(client.update_clientid, 'value')

display(clients_mgmt)

## Configure Nano's Work Orders (WO) types

In [ ]:
button = widgets.Button(description="Configure WO",
                        layout=widgets.Layout(width='95px', 
                                              height='60px'))

button.on_click(client.config_wo)

display(button)


update_wo_types = widgets.Button(description='Update all\nWO types',
                                 layout=widgets.Layout(width='200px', 
                                                       height='80px'))
display(update_wo_types)
    
update_wo_types.on_click(client.update_wo_call,)

## Compute Time To Fail

In [ ]:
compute_ttf = widgets.Button(description='Compute TTF',
                             layout=widgets.Layout(width='95px', 
                                                   height='60px'))
display(compute_ttf)
    
compute_ttf.on_click(client.compute_ttf,)

In [ ]:
save_ttf = widgets.Button(description='Save TTF',
                                 layout=widgets.Layout(width='95px', 
                                                       height='60px'))
display(save_ttf)
    
save_ttf.on_click(client.save_json,)

In [ ]:
print(client._ttf)